## 変分アルゴリズム

このコースでは、変分アルゴリズムの詳細と、量子力学の変分定理に基づく近い将来のハイブリッド量子-古典アルゴリズムについて説明します。これらのアルゴリズムは、今日のフォールトトレラントでない量子コンピューターによって提供されるユーティリティを活用できるため、[量子優位性](gloss:advantage)を達成するための理想的な候補になります。

このコースでは次のことを学びます。

- 変分アルゴリズム設計ワークフローの各ステップ
- 各ステップに関連するトレードオフ
- [Qiskit Runtimeプリミティブ](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)を使用して速度と精度を最適化する方法

このコースは、研究者や開発者が量子コンピューターの有用性を探求するための出発点となることを意図していますが、量子コンピューティング全般の理論的・基礎的な知識は[量子情報と計算の基礎](https://qiskit.org/learn/course/basics-quantum-information/)( [YouTubeシリーズ](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqKKMXX4vbi4EB1uaErFMSO)としても視聴可能）を自由に探検してみてください。

## 簡素化されたハイブリッドワークフロー

![Variational Flow](images/variational_workflow.png)

変分アルゴリズムには、アルゴリズム、ソフトウェア、およびハードウェアの進歩に基づいて組み合わせて最適化できるいくつかのモジュラーコンポーネントが含まれています。これには、一連のパラメーターを使用して特定の問題を記述する*コスト関数*、これらのパラメーターを使用して探索空間を表現するための*ansatz* 、および探索空間を繰り返し探索するための*オプティマイザー*が含まれます。各反復中に、オプティマイザーは現在のパラメーターを使用してコスト関数を評価し、最適解に[収束](gloss:converge)するまで次の反復のパラメーターを選択します。この種のアルゴリズムがハイブリッドな点は、コスト関数が量子リソースを使用して評価され、古典リソースによって最適化されることに由来します。

1. **問題の初期化**: 変分アルゴリズムは、量子コンピューターを*初期状態* $|0\rangle$ に初期化することから始まり、次にそれを望みの (パラメーター化されていない) 状態 $|\rho\rangle$ に変換します。これを*参照状態 *と呼びます。

    この変換は、 $U_R|0\rangle = |\rho\rangle$ のように、既定の状態にユニタリ参照演算子 $U_R$ を適用することで表現されます。

2. **ansatzの準備**: 初期状態 $|0\rangle$ からターゲット状態 $|\psi(\vec\theta)\rangle$ への反復最適化を開始するには、*変分形式* $U_V(\vec\theta)$ を定義する必要があります。変分アルゴリズムが探索するパラメータ化された状態のコレクションを表します。

    参照状態と変分形式の特定の組み合わせをansatzと呼びます。例えば $U_A(\vec\theta) := U_V(\vec\theta) U_R$ です。<br>ansatzは最終的に、初期状態 $|0\rangle$ をターゲット状態 $|\psi(\vec\theta)\rangle$ に変換できるパラメータ化された量子回路の形を取ります。

    全体として次のようになります。

   $$
   \begin{aligned}
   |0\rangle \xrightarrow{U_R} U_R|0\rangle

   & = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle \\[1mm]

   & = U_V(\vec{\theta})U_R|0\rangle \\[1mm]

   & = U_V(\vec{\theta})|\rho\rangle \\[1mm]

   & = |\psi(\vec{\theta})\rangle \\[1mm]

   \end{aligned}
   $$

3. **コスト関数の評価**: 問題を*コスト関数* $C(\vec\theta)$ にエンコードし、量子系で実行するパウリ演算子の線形結合として使用できます。これは、エネルギーやスピンなどの物理系に関する情報である可能性がありますが、物理でない問題も同じようにエンコードすることができます。 Qiskit Runtimeプリミティブを活用して、コスト関数を評価しながらエラーの抑制と軽減でノイズに対処できます。

4. **パラメーターの最適化**: 評価結果は古典コンピューターに送られ、そこで古典オプティマイザーが分析し、変分パラメーターの次の値のセットを選択します。既存の最適解がある場合、それを*初期点* $\vec\theta_0$ として設定して、最適化を*ブートストラップ*できます。この*初期状態* $|\psi(\vec\theta_0)\rangle$ を使用すると、オプティマイザーが有効な解をより速く見つけるのに役立ちます。

5. **結果に基づいた ansatz パラメーターの調整と再実行**: 古典オプティマイザーの終端基準が満たされるまでプロセス全体が繰り返され、パラメーター値の最適なセット $\vec\theta^ {em1}$ が返されます。そして、この問題の解の状態は、 $|\psi(\vec\theta^ )\rangle = U_A(\vec\theta^*)|0\rangle$ と提案されます。

## 変分定理

変分アルゴリズムの一般的な目標は、特定の観測可能量の最小または最大の固有値を持つ量子状態を見つけることです。ここで使用する重要な洞察は、量子力学の*変分定理*です。詳細に入る前に、その背後にある数学的直感をいくつか調べてみましょう.

### エネルギーと基底状態の数学的直感

量子力学では、エネルギーは通常*ハミルトニアン*と呼ばれる観測可能量の形で表され、これを $\hat{\mathcal{H}}$ で表します。[スペクトル分解](gloss:decomposition)を考えてみましょう :

$$
\hat{\mathcal{H}} = \sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|
$$

ここで、 $N$ は状態空間の次元、 $\lambda_{k}$ は $k$ 番目の固有値、または物理的には $k$ 番目のエネルギー準位、 $|\phi_k\rangle$ は $\hat{\mathcal{H}}|\phi_k\rangle = \lambda_k |\phi_k\rangle$ に対応する[固有状態](gloss:eigenstate):  (正規化された) 状態 $|\psi\rangle$ の系の期待エネルギーは以下のようになります。

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \langle \psi |\bigg(\sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|\bigg) | \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k \langle \psi |\phi_k\rangle \langle \phi_k| \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

\end{aligned}
$$

$\lambda_0\leq \lambda_k, \forall k$ を考慮すると、次のようになります。

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& \geq  \sum_{k=0}^{N-1} \lambda_0 |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \\[1mm]

\end{aligned}
$$

`{latex} \{ |\phi_k\rangle \}_{k=0}^{N-1}` は正規直交基底であるため、 $|\phi_{k} \rangle$ を測定する確率は $p_k = |\langle \psi |\phi_{k} \rangle |^2$ であり、すべての確率の合計は $\sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 = \sum_{k=0}^{N-1}p_k = 1$ となります。要するに、系の期待されるエネルギーは、最小エネルギーまたは基底状態エネルギーよりも高くなります。

$$
\langle \psi | \hat{\mathcal{H}} | \psi \rangle \geq \lambda_0.
$$

上記の引数は、有効な (正規化された) 量子状態 $|\psi\rangle$ に適用されるため、パラメーターベクトル $\vec\theta$ に依存するパラメーター化された状態 $|\psi(\vec\theta)\rangle$ を考慮することは完全に可能です。ここで「変分」の出番です。 $C(\vec\theta) := \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle$ で与えられるコスト関数を考えるとそれを最小化したい場合、最小値は常に次を満たします。

$$
\min_{\vec\theta} C(\vec\theta) = 
\min_{\vec\theta} \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle \geq \lambda_0.
$$

$C(\vec\theta)$ の最小値は、パラメータ化された状態 $|\psi(\vec\theta)\rangle$ を使用して $\lambda_0$ に到達できる最も近い値になり、 $|\psi(\vec\theta^{em0})\rangle = |\phi_0\rangle$ となるパラメータベクトル $\vec\theta^{/em0}$ が存在する場合にのみ等号が成り立ちます。

### 量子力学の変分定理

量子系の (正規化された) 状態 $|\psi\rangle$ がパラメータベクトル $\vec\theta$ に依存する場合、基底状態の最適近似 (つまり、最小固有値 $\lambda_0$ をとる固有状態 $|\phi_0\rangle$) は、ハミルトニアン $\hat{\mathcal{H}}$ の[期待値](gloss:expectation)を最小化するものです:

$$
\langle \hat{\mathcal{H}} \rangle(\vec\theta) := 
\langle \psi(\vec\theta) |\hat{\mathcal{H}}| \psi(\vec\theta) \rangle \geq 
\lambda_0
$$

変分定理がエネルギー最小値の観点から述べられている理由は、多くの数学的仮定が含まれているためです。

- 物理的な理由から、 $N\rightarrow\infty$ であっても、エネルギー  $E \geq \lambda_0 > -\infty$ を満たす有限の下限が存在する必要があります。
- 通常上限は存在しません。

ただし、数学的に言えば、ハミルトニアン $\hat{\mathcal{H}}$ についてこれらの仮定を超える特別なことは何もないため、定理は他の観測可能量とその固有状態に一般化できますが、同じ制約に従う場合に限ります。また、有限の上限が存在する場合、下限を上限に交換することにより、固有値を最大化するために同じ数学的議論を行うことができることに注意してください。

このレッスンでは、変分アルゴリズムの概要を学習しました。以降のレッスンでは、各ステップとそれに関連するトレードオフについて詳しく説明します。